In [1]:
# !pip install kaggle

In [2]:
# ! mkdir ~/.kaggle

In [3]:
# ! cp kaggle.json ~/.kaggle

In [4]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:49<00:00, 23.5MB/s]
100% 1.06G/1.06G [00:49<00:00, 22.9MB/s]


In [5]:
# !unzip -qq dogs-vs-cats.zip

In [6]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout

In [7]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256)
)

Found 20000 files belonging to 2 classes.


In [8]:
test_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256)
)

Found 5000 files belonging to 2 classes.


In [9]:
def process(image, label):
  image = tf.cast(image/255., tf.float32)
  return image, label

train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

In [10]:
model = Sequential()

In [11]:
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

In [12]:
model.add(Conv2D(filters=64, kernel_size=3, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

In [13]:
model.add(Conv2D(filters=128, kernel_size=3, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

In [14]:
model.add(Flatten())

In [15]:
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1, activation='sigmoid'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 254, 254, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 125, 125, 64)      256       
 chNormalization)                                                
                                                        

In [19]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [20]:
history = model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10
625/625 [==============================] - 77s 107ms/step - loss: 1.3049 - accuracy: 0.5883 - val_loss: 0.6056 - val_accuracy: 0.6718
Epoch 2/10
625/625 [==============================] - 65s 104ms/step - loss: 0.5656 - accuracy: 0.7028 - val_loss: 0.5747 - val_accuracy: 0.6664
Epoch 3/10
625/625 [==============================] - 68s 109ms/step - loss: 0.4914 - accuracy: 0.7588 - val_loss: 0.5756 - val_accuracy: 0.6830
Epoch 4/10
625/625 [==============================] - 68s 109ms/step - loss: 0.4379 - accuracy: 0.8019 - val_loss: 0.5150 - val_accuracy: 0.7668
Epoch 5/10
625/625 [==============================] - 66s 104ms/step - loss: 0.3911 - accuracy: 0.8291 - val_loss: 0.5149 - val_accuracy: 0.7962
Epoch 6/10
625/625 [==============================] - 65s 104ms/step - loss: 0.3144 - accuracy: 0.8681 - val_loss: 0.4568 - val_accuracy: 0.7914
Epoch 7/10
625/625 [==============================] - 65s 103ms/step - loss: 0.2245 - accuracy: 0.9093 - val_loss: 0.6055 - val_ac

In [21]:
model.save("/content/dogs_vs_cat.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [73]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.utils import load_img, img_to_array
test_image = load_img('/content/test/dogs/dog.1002.jpg', target_size=(256, 256))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

test_image = test_image / 255.0

result = model.predict(test_image)

if result[0][0] < 0.5:
    prediction = 'cat'
else:
    prediction = 'dog'

1/1 [==============================] - 0s 20ms/step


In [74]:
print(prediction)

dog
